## LIBRARIES being used on this project

In [27]:
import pandas as pd

## Extract, Transform, Load (ETL)
Data link: https://www.openml.org/search?type=data&sort=runs&id=43430&status=active

If within the dataset, there's "@"/"%"/"" than skip, if not, append into data.

In [33]:
data = []

with open(r"../data/dataset", "r") as x: #'r'-read mode/ 'w'-write/ 'rb'- read binary/ 'a' - append
    for line in x.read().split("\n"):
        if line.startswith("@") or line.startswith("%") or line == "":
            continue
        data.append(line)

Now, to gather the attributes (columns), we look for the second word.


<img alt="variables_example" height="400" src="images/attributes.png" width="400"/>

In [30]:
columns = []

with open(r"../data/dataset", "r") as x:
    for line in x.read().split("\n"):
        if line.startswith("@ATTRIBUTE"):
            columns.append(line.split(" ")[1])

Writing everything into a .csv file and saving in the data folder.

In [36]:
with open("../data/df.csv", "w") as x:
    x.write(",".join(columns))
    x.write("\n")
    x.write("\n".join(data))

read the dataframe with pandas.

In [38]:
df = pd.read_csv("../data/df.csv")

## Exploratory data analysis (EDA)

In [42]:
df.head(4)

,bomb_planted,ct_armor,ct_defuse_kits,ct_grenade_decoygrenade,ct_grenade_flashbang,ct_grenade_hegrenade,ct_grenade_incendiarygrenade,ct_grenade_molotovgrenade,ct_grenade_smokegrenade,ct_health,...,t_weapon_r8revolver,t_weapon_sawedoff,t_weapon_scar20,t_weapon_sg553,t_weapon_ssg08,t_weapon_tec9,t_weapon_ump45,t_weapon_usps,t_weapon_xm1014,time_left
0,175.00,0.0,0.0,de_dust2,False,500.0,500.0,0.0,0.0,4000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CT
1,156.03,0.0,0.0,de_dust2,False,500.0,500.0,400.0,300.0,600.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,CT
2,96.03,0.0,0.0,de_dust2,False,391.0,400.0,294.0,200.0,750.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,CT
3,76.03,0.0,0.0,de_dust2,False,391.0,400.0,294.0,200.0,750.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CT


In [44]:
df.tail(4)

,bomb_planted,ct_armor,ct_defuse_kits,ct_grenade_decoygrenade,ct_grenade_flashbang,ct_grenade_hegrenade,ct_grenade_incendiarygrenade,ct_grenade_molotovgrenade,ct_grenade_smokegrenade,ct_health,...,t_weapon_r8revolver,t_weapon_sawedoff,t_weapon_scar20,t_weapon_sg553,t_weapon_ssg08,t_weapon_tec9,t_weapon_ump45,t_weapon_usps,t_weapon_xm1014,time_left
122406,174.93,11.0,15.0,de_train,False,500.0,500.0,95.0,175.0,11500.0,...,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,T
122407,114.93,11.0,15.0,de_train,False,500.0,500.0,495.0,475.0,1200.0,...,4.0,3.0,5.0,1.0,0.0,0.0,5.0,0.0,0.0,T
122408,94.93,11.0,15.0,de_train,False,500.0,500.0,495.0,475.0,1200.0,...,5.0,0.0,3.0,0.0,0.0,0.0,4.0,0.0,0.0,T
122409,74.93,11.0,15.0,de_train,False,375.0,479.0,395.0,466.0,1100.0,...,3.0,0.0,2.0,0.0,0.0,0.0,3.0,0.0,0.0,T


In [45]:
df.shape

(122410, 97)

In [46]:
df.describe()

,bomb_planted,ct_armor,ct_defuse_kits,ct_grenade_hegrenade,ct_grenade_incendiarygrenade,ct_grenade_molotovgrenade,ct_grenade_smokegrenade,ct_health,ct_helmets,ct_money,...,t_weapon_p90,t_weapon_r8revolver,t_weapon_sawedoff,t_weapon_scar20,t_weapon_sg553,t_weapon_ssg08,t_weapon_tec9,t_weapon_ump45,t_weapon_usps,t_weapon_xm1014
count,122410.000000,122410.000000,122410.000000,122410.000000,122410.000000,122410.000000,122410.000000,122410.000000,122410.000000,122410.000000,...,122410.000000,122410.000000,122410.000000,122410.000000,122410.000000,122410.000000,122410.000000,122410.000000,122410.000000,122410.000000
mean,97.886922,6.709239,6.780435,412.106568,402.714500,314.142121,298.444670,9789.023773,11241.036680,2.053901,...,1.853157,1.858100,1.540814,1.627146,1.001969,0.019819,0.048011,1.352095,0.027694,0.025750
std,54.465238,4.790362,4.823543,132.293290,139.919033,171.029736,174.576545,11215.042286,12162.806759,1.841470,...,1.772791,1.794473,1.737804,1.829147,1.458084,0.143933,0.227669,1.663246,0.169531,0.164162
min,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,54.920000,3.000000,3.000000,350.000000,322.000000,194.000000,174.000000,1300.000000,1550.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,94.910000,6.000000,6.000000,500.000000,500.000000,377.000000,334.000000,5500.000000,7150.000000,2.000000,...,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
75%,166.917500,10.000000,10.000000,500.000000,500.000000,486.000000,468.000000,14600.000000,18000.000000,4.000000,...,3.000000,3.000000,3.000000,3.000000,2.000000,0.000000,0.000000,2.000000,0.000000,0.000000
max,175.000000,32.000000,33.000000,500.000000,600.000000,500.000000,500.000000,80000.000000,80000.000000,5.000000,...,7.000000,7.000000,6.000000,9.000000,5.000000,3.000000,3.000000,5.000000,3.000000,2.000000


There's no null value in the DataFrame

In [47]:
df.isnull().sum()

bomb_planted               0
ct_armor                   0
ct_defuse_kits             0
ct_grenade_decoygrenade    0
ct_grenade_flashbang       0
                          ..
t_weapon_tec9              0
t_weapon_ump45             0
t_weapon_usps              0
t_weapon_xm1014            0
time_left                  0
Length: 97, dtype: int64